In [ ]:
# import
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
import numpy as np
import os

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Normalize pixel values
x_train, x_test = x_train / 255.0, x_test / 255.0

# Convert labels to one-hot encoding
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)
datagen.fit(x_train)

# Callbacks for better training
checkpoint = ModelCheckpoint("cifar10_best_model.h5", monitor="val_accuracy", save_best_only=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=5, verbose=1)
early_stopping = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True, verbose=1)

# Define a deeper CNN model inline
model = models.Sequential([
    # Block 1
    layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)),
    layers.BatchNormalization(),
    layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),

    # Block 2
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),

    # Block 3
    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),

    # Extra Block 4 for deeper features
    layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),

    # Fully Connected Layers (ann)
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train Model
history = model.fit(
    datagen.flow(x_train, y_train, batch_size=64),
    epochs=100,  # Adjust if needed
    validation_data=(x_test, y_test),
    callbacks=[checkpoint, reduce_lr, early_stopping]
)

# Save final model after training
model.save("cifar10_final_model.h5")

# Evaluate the model
loss, acc = model.evaluate(x_test, y_test, verbose=2)
print(f"Test accuracy after training: {acc * 100:.2f}%")


Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.2814 - loss: 2.1205
Epoch 1: val_accuracy improved from -inf to 0.47680, saving model to cifar10_best_model.h5


782/782 ━━━━━━━━━━━━━━━━━━━━ 58s 49ms/step - accuracy: 0.2815 - loss: 2.1200 - val_accuracy: 0.4768 - val_loss: 1.4507 - learning_rate: 0.0010
Epoch 2/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.4966 - loss: 1.4165
Epoch 2: val_accuracy improved from 0.47680 to 0.59080, saving model to cifar10_best_model.h5


782/782 ━━━━━━━━━━━━━━━━━━━━ 26s 34ms/step - accuracy: 0.4967 - loss: 1.4164 - val_accuracy: 0.5908 - val_loss: 1.1479 - learning_rate: 0.0010
Epoch 3/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.5879 - loss: 1.1816
Epoch 3: val_accuracy improved from 0.59080 to 0.61380, saving model to cifar10_best_model.h5


...

782/782 ━━━━━━━━━━━━━━━━━━━━ 26s 33ms/step - accuracy: 0.9227 - loss: 0.2225 - val_accuracy: 0.9018 - val_loss: 0.3176 - learning_rate: 2.5000e-04
Epoch 45/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.9245 - loss: 0.2172
Epoch 45: val_accuracy did not improve from 0.90180
782/782 ━━━━━━━━━━━━━━━━━━━━ 26s 33ms/step - accuracy: 0.9245 - loss: 0.2172 - val_accuracy: 0.8909 - val_loss: 0.3563 - learning_rate: 2.5000e-04
Epoch 46/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.9209 - loss: 0.2338
Epoch 46: val_accuracy did not improve from 0.90180
782/782 ━━━━━━━━━━━━━━━━━━━━ 26s 33ms/step - accuracy: 0.9209 - loss: 0.2338 - val_accuracy: 0.8964 - val_loss: 0.3357 - learning_rate: 2.5000e-04
Epoch 47/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.9230 - loss: 0.2191
Epoch 47: val_accuracy did not improve from 0.90180
782/782 ━━━━━━━━━━━━━━━━━━━━ 25s 33ms/step - accuracy: 0.9230 - loss: 0.2191 - val_accuracy: 0.8964 - val_loss: 0.3393 - learning_rate: 

782/782 ━━━━━━━━━━━━━━━━━━━━ 26s 33ms/step - accuracy: 0.9328 - loss: 0.1865 - val_accuracy: 0.9035 - val_loss: 0.3183 - learning_rate: 1.2500e-04
Epoch 56/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.9326 - loss: 0.1916
Epoch 56: val_accuracy did not improve from 0.90350
782/782 ━━━━━━━━━━━━━━━━━━━━ 26s 33ms/step - accuracy: 0.9326 - loss: 0.1917 - val_accuracy: 0.8971 - val_loss: 0.3445 - learning_rate: 1.2500e-04
Epoch 57/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.9361 - loss: 0.1867
Epoch 57: val_accuracy did not improve from 0.90350
782/782 ━━━━━━━━━━━━━━━━━━━━ 26s 33ms/step - accuracy: 0.9361 - loss: 0.1867 - val_accuracy: 0.9016 - val_loss: 0.3293 - learning_rate: 1.2500e-04
Epoch 58/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.9352 - loss: 0.1845
Epoch 58: val_accuracy improved from 0.90350 to 0.90370, saving model to cifar10_best_model.h5


782/782 ━━━━━━━━━━━━━━━━━━━━ 26s 33ms/step - accuracy: 0.9352 - loss: 0.1845 - val_accuracy: 0.9037 - val_loss: 0.3221 - learning_rate: 1.2500e-04
Epoch 59/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.9323 - loss: 0.1899
Epoch 59: val_accuracy did not improve from 0.90370

Epoch 59: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.
782/782 ━━━━━━━━━━━━━━━━━━━━ 26s 33ms/step - accuracy: 0.9323 - loss: 0.1899 - val_accuracy: 0.9011 - val_loss: 0.3370 - learning_rate: 1.2500e-04
Epoch 60/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.9389 - loss: 0.1753
Epoch 60: val_accuracy did not improve from 0.90370
782/782 ━━━━━━━━━━━━━━━━━━━━ 25s 32ms/step - accuracy: 0.9389 - loss: 0.1753 - val_accuracy: 0.9019 - val_loss: 0.3266 - learning_rate: 6.2500e-05
Epoch 61/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.9395 - loss: 0.1742
Epoch 61: val_accuracy did not improve from 0.90370
782/782 ━━━━━━━━━━━━━━━━━━━━ 26s 33ms/step - accuracy: 0.93

782/782 ━━━━━━━━━━━━━━━━━━━━ 26s 33ms/step - accuracy: 0.9389 - loss: 0.1720 - val_accuracy: 0.9042 - val_loss: 0.3206 - learning_rate: 6.2500e-05
Epoch 63/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.9385 - loss: 0.1716
Epoch 63: val_accuracy did not improve from 0.90420
782/782 ━━━━━━━━━━━━━━━━━━━━ 26s 33ms/step - accuracy: 0.9385 - loss: 0.1716 - val_accuracy: 0.9037 - val_loss: 0.3213 - learning_rate: 6.2500e-05
Epoch 64/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.9381 - loss: 0.1766
Epoch 64: val_accuracy did not improve from 0.90420

Epoch 64: ReduceLROnPlateau reducing learning rate to 3.125000148429535e-05.
782/782 ━━━━━━━━━━━━━━━━━━━━ 26s 33ms/step - accuracy: 0.9381 - loss: 0.1766 - val_accuracy: 0.9012 - val_loss: 0.3351 - learning_rate: 6.2500e-05
Epoch 64: early stopping
Restoring model weights from the end of the best epoch: 54.


313/313 - 1s - 3ms/step - accuracy: 0.9017 - loss: 0.3146
Test accuracy after training: 90.17%
